In [ ]:
import chargeback_rpt.db_postgres_command as db_command
import chargeback_rpt.vm_data_utility as vm_util

import chargeback_rpt.vm_data_validator as vx
import chargeback_rpt.file_directory_manager as fd_mn
import chargeback_rpt.email_notifier as x_mail


import pandas as pd
import numpy as np
import datetime as datetime
import hpestorapi;

import random
import sys 

In [ ]:
x_ip = '0.0.0.0'
x_username = 'abc'
x_password = 'xyz'

#https://stackoverflow.com/questions/5194057/better-way-to-convert-file-sizes-in-python
x_convert_unit=30  # convert bytes to gb
x_unit_str='GB'
#x_convert_unit=40  # convert bytes to tb




In [ ]:
col_name='name'
col_createdDate='createdDate'
col_userBytes='userBytes'
col_diskBytes='diskBytes'


tb_key="storeonce_table"
# catalyst_name  user_size   disk_size  reated_date month  year import_date



In [ ]:
run_py=False
fix_mmyy=False
is_only_check_data=False

x_datenow=datetime.datetime.now()
month_x=x_datenow.strftime('%m')
year_x=x_datenow.strftime('%Y')

if fix_mmyy:
 month_param='09'
 year_param='2022'
else:
 month_param=month_x
 year_param=year_x
    
list_error=[]
print(list_error)

print(f"{month_param}-{year_param}")

In [ ]:
if  run_py:

    press_y='n'
    ok=False
    
    if len(sys.argv) > 1:
        if sys.argv[1]=='0'  and sys.argv[2]=='0':
         month_param=month_x
         year_param=year_x 
         ok=True   
         is_only_check_data=bool(int(sys.argv[3]))
        else:
         raise Exception("Specify 0 0 as arguments only")   
    else:
        print("If you want current month and year.Press Enter ")
        month_param = int(input("Enter the month (1-12) : ") or month_x)
        year_param = int(input("Enter the year(such as 2021,2022,2023...) : ") or year_x)
        print(f"Do for month={month_param} and year={year_param}")
        press_y=input(f"Press y=True and n=False : ") 
        if press_y.lower()=='y':
         ok=True


    if ok==True:
        try:
         report_dt= datetime.datetime(int(year_param), int(month_param), 1)
         print(report_dt)   
         month_param=report_dt.strftime('%m')
         year_param=report_dt.strftime('%Y')   


        except Exception as ex:
         print(ex)   
         raise ex
    else:
         quit()

In [ ]:
# Move  to util
def check_error_point_etl(tran_id):
  print(list_error)
  if True in list_error:
        
    vm_util.collect_error_to_sent_mail(tran_id) 
    
    print("ETL occured error") 
    
    raise Exception("Program is teminated and check error from email and log_error.txt")
    
  list_error.clear()
  print(list_error)   

# Start Transaction

In [ ]:
t_id=vm_util.creating_transaction(13,month_param,year_param)
print(f"ETL Transaction ID: {t_id}")

In [ ]:
storeonce_prefex_site_name=vm_util.get_config_value('storeonce_prefex_site_name',t_id)

# Get  data from StoreOnce 

In [ ]:
print("get  data from StoreOnce and enrich data to dataframe")
def getCatalystInfo():
    
 try:  
    with hpestorapi.StoreOnceG4(x_ip, x_username, x_password) as so:
        so.open()
        status, data = so.get('/api/v1/data-services/cat/stores')
        if status == 200:
         
         df_x=None
         list_catalystName=[]   
         list_createdDate=[]
         list_userSize=[]
         list_diskSize=[]
        
#          for x in ['BPK-Catalyst'] :        
                
         for group in data['members']:

            catalystName=group['name']
            catalystName=catalystName.replace(storeonce_prefex_site_name,'')
            createdDate=group['createdDate']              #convert date as yyyy-mm-ddd format
            userBytes=group['userBytes']
            userSize=float(userBytes)/float(1<<x_convert_unit)
            diskBytes=group['diskBytes']
            diskSize=float(diskBytes)/float(1<<x_convert_unit)

            print(f'{catalystName} - {createdDate}')
            print(f'Total Disk (userBytes) ={userBytes} Bytes  =  {userSize} {x_unit_str}')
            print(f'ActualUsage Disk (diskBytes) ={diskBytes} Bytes  =  {diskSize} {x_unit_str}')
            print("=========================================================")
            
            # Test Smaple Data
#             catalystName=x.replace(storeonce_prefex_site_name,'')
#             createdDate=datetime.datetime.now()
#             userSize=random.randint(1000, 10000)
#             diskSize=round(userSize/2)
    
            list_catalystName.append(catalystName)   
            list_createdDate.append(createdDate)
            list_userSize.append(userSize)   
            list_diskSize.append(diskSize)      
                  
            if  len(list_catalystName)>0:
               df_x=pd.DataFrame(data={'catalyst_name':list_catalystName,'created_date':list_createdDate,'user_size':list_userSize,'disk_size':list_diskSize})
            
            # Incorperate Addtional Data
            df_x['month']=month_param
            df_x['year']=year_param
            df_x['import_date']=x_datenow
            
         return df_x  
   
 except Exception as error:
        error_message=f'Cannot connect to {x_ip} because {str(error)}'
        list_error.append(True)
        print(error_message)   
        vm_util.add_error_to_database(14,error_message,t_id)  

            
              
# 'sizeOnDiskQuotaBytes': 214748364800,
# 'userDataStoredQuotaBytes': 4398046511104,
# 'userBytes': 5856889966,
# 'diskBytes': 3592012364,



In [ ]:
df_catalyst=getCatalystInfo()
check_error_point_etl(t_id)



In [ ]:
print(df_catalyst.info())
df_catalyst


# Save to database

In [ ]:
print("save  dataframe to database")
table_name,listCols_report,is_error=vx.listCols_report_table(tb_key,t_id)
list_error.append(is_error)


print(f'list all columns in {table_name} table: ',len(listCols_report))
print(listCols_report)

listCols_dfx=df_catalyst.columns.tolist()
print(f'list all columns in {table_name} dataframe: ',len(listCols_dfx))
print(listCols_dfx)


check_columns_2=vm_util.verify_existing_columns(listCols_dfx,listCols_report)

print(check_columns_2)


print("Save data to database")
if check_columns_2  is None:
   df_catalyst=df_catalyst[listCols_report]
   print(df_catalyst.info())
   print(df_catalyst.head())
   rslt=db_command.add_data_values(db_command.get_postgres_conn(),df_catalyst,table_name)
   print(rslt)
    
else:
   list_error.append(True)
   error_message=check_columns_2
   vm_util.add_error_to_database(10,error_message,t_id)
   print(error_message)
    
check_error_point_etl(t_id)

# Check New Site For StoreOne

In [ ]:
def add_new_site(siteList,tran_id):
   
   df_new_site = None
   try:

        all_cost_center_key = 'email_link_all_site_storeonce'
        update_cost_center_key = 'email_link_update_site_storeonce'

        new_ccList = []

        def insert_new_site(cc_name):
            try:
                cc_name_low = cc_name.replace(' ', '').lower()
                sql = "select site_name from site_info where lower(replace(site_name, ' ', '')) = (%s) "
                param_x = (cc_name_low,)
                cc_item = db_command.get_one_sql(db_command.get_postgres_conn(), sql, param_x)

                if cc_item is None:
                    print('new site : ', cc_name)
                    params = (cc_name, '-', '-', '-','-','-','-')
                    sql_insert = """
                    INSERT INTO site_info(site_name,site_full_name,site_address,site_owner,site_owner_email,site_approver_name,site_approver_position) 
                    VALUES(%s,%s,%s,%s,%s,%s,%s) RETURNING id;
                    """
                    new_cc_id = db_command.add_one_data_sql(db_command.get_postgres_conn(), sql_insert, params)
                    
                    new_ccList.append([new_cc_id, cc_name])

            except Exception as ex:
                error_message = str(ex)
                print(error_message)
                add_error_to_database(12, error_message, tran_id)
                raise ex

        def collect_new_costcenter_to_sent_mail():
            #mail_new_site_template.html
            try:
                link_all_cc = vm_util.get_config_value(all_cost_center_key, tran_id)
                link_update_cc = vm_util.get_config_value(update_cost_center_key, tran_id)

                content_data_dict = {
                    "ContentTitle": f"List New Site For StoreOnce",
                    "New_Site_StoreOnce": df_new_site,
                    "Len_Cols_New_Site_StoreOnce": len(df_new_site.columns),
                    'LinkAll': link_all_cc,
                    'LinkEach': link_update_cc
                }
                ok = x_mail.send_email(email_type='new_site_catalyst', transaction_id=tran_id,
                                       attached_file_path=None, content_data_dict=content_data_dict)
#                 ok=True
                return ok

            except Exception as ex:
                print(str(ex))
                raise ex


        for item in siteList:
            insert_new_site(item)

        if len(new_ccList) > 0:
            df_new_site = pd.DataFrame(new_ccList, columns=['ID', 'SiteName'])
            #collect_new_costcenter_to_sent_mail()
             
   except Exception as ex:
        raise ex


   return df_new_site

In [ ]:
print("Update new site")

try :
 siteList = df_catalyst['catalyst_name'].unique()
 df_new_cc=add_new_site(siteList,t_id)
except Exception as ex:
 list_error.append(True)   
 print(ex)   

check_error_point_etl(t_id)


# Complete Transaction

In [ ]:
try: 
  
     updated_rows=vm_util.created_transaction(t_id)
     print("completed ETL")
    
except Exception as ex:
    list_error.append(True)
    print(ex)
    
check_error_point_etl(t_id)